# check ponding depth and cumulative infiltration for simulations with impervious bottom BC

This notebook allows to estimate the ponding depth at soil surface and the cumulative infiltration for a soil of known SWRC parameters, cumulative rainfall heights, and with impervious bottom.

From the input file (.nc) it is possible to compute the total volume of air within the soil that is available to infiltration. To compute this volume the necessary data are
- intial condition of water head (psiIC)
- SWRC model and its parameters (par1SWRC, par2SWRC, thetaS, and thetaR)
All these information are contained in the input file (.nc)

From the output file (.nc) it is possible to know the cumulative rainfall height (from variable topBC), and the water ponding depth.

Since the bottom is impervious every single rain drop will remain within the domain. Thus, perfomorming a simple mass budget it is possible to compute the water ponding depth as:

$$estimatedPondigDepth = cumulativeRainfall - totalAirVolume$$

and compare this with that computed by making use of Richards1D solver

In [1]:
"""
  GNU GPL v3 License
 
  Copyright 2018 Niccolo` Tubini
 
  This program is free software: you can redistribute it and/or modify
  it under the terms of the GNU General Public License as published by
  the Free Software Foundation, either version 3 of the License, or
  (at your option) any later version.
 
  This program is distributed in the hope that it will be useful,
  but WITHOUT ANY WARRANTY; without even the implied warranty of
  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
  GNU General Public License for more details.
 
  You should have received a copy of the GNU General Public License
  along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

"""
Created on Tue Apr 17 14:52:22 2018

@author: Niccolo` Tubini

This code was obtained modifing the code pres_temp_4D_rd.py 
by Jeff Whitaker <jeffrey.s.whitaker@noaa.gov> 20070202
https://www.unidata.ucar.edu/software/netcdf/examples/programs/

"""

#from netCDF4_classic import Dataset
from netCDF4 import Dataset


import os
import math  
## pandas
import pandas as pd

## numpy
import numpy as np

## plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
%matplotlib inline
import matplotlib.transforms as transforms
import matplotlib.ticker as mtick


## Read input file
The grid file allows to compute the volume of air, available for infiltration

In [4]:
os.chdir("C:\\Users\\Niccolo\\OMS\\OMS_Project_Richards1D_Newest\\data\\Grid_NetCDF")
os.listdir()

['ClaySand_noPonding.nc',
 'Clay_noPonding.nc',
 'SandClay_01Ponding_Measurement.nc',
 'Sand_01Ponding.nc']

In [5]:
##########
##########

## open netCDF file for reading.

ncfile = Dataset('Clay_noPonding.nc','r') 

print ('*** SUCCESS reading')

print('\n ***FILE INFO:\n')
print(ncfile)

## other usefull commands:
#print (ncfile.dimensions['time'])
#print (ncfile.file_format)
#print (ncfile.dimensions.keys())
print (ncfile.variables.keys())
#print (ncfile.variables['psi'])

psiIC = ncfile.variables['psiIC']
print('\n ***psiIC INFO:\n')
print(psiIC)

thetaS = ncfile.variables['thetaS']
print('\n ***thetaS INFO:\n')
print(thetaS)

thetaR = ncfile.variables['thetaR']
print('\n ***thetaR INFO:\n')
print(thetaR)

par1SWRC = ncfile.variables['par1SWRC']
print('\n ***par1SWRC INFO:\n')
print(par1SWRC)

par2SWRC = ncfile.variables['par2SWRC']
print('\n ***par2SWRC INFO:\n')
print(par2SWRC)

deltaZ = ncfile.variables['deltaZ']
print('\n ***deltaZ INFO:\n')
print(deltaZ)

*** SUCCESS reading

 ***FILE INFO:

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: Grid 1 layer clay with no ponding water, hydrostatic, Clay_noPonding_STORATIVITY_test.csv
    institution: Geoframe
    summary: This file stores all grid information (geometry, soil properties initial condition)
    date_created: 5/14/2018
    dimensions(sizes): z(321), zz(320)
    variables(dimensions): float64 eta(z), float64 etaDual(z), float64 z(z), float64 zDual(z), float64 psiIC(z), float64 spaceDelta(z), float64 et(z), float64 deltaZ(zz), float64 thetaS(zz), float64 thetaR(zz), float64 Ks(zz), float64 par1SWRC(zz), float64 par2SWRC(zz), float64 par3SWRC(zz), float64 par4SWRC(zz)
    groups: 

odict_keys(['eta', 'etaDual', 'z', 'zDual', 'psiIC', 'spaceDelta', 'et', 'deltaZ', 'thetaS', 'thetaR', 'Ks', 'par1SWRC', 'par2SWRC', 'par3SWRC', 'par4SWRC'])

 ***psiIC INFO:

<class 'netCDF4._netCDF4.Variable'>
float64 psiIC(z)
    units: m
    long_name: i

In [6]:
def VG(psi,n,alpha,thetaS,thetaR):
    """(float, float, float, float) -> float 
    
       This function returns the saturation degree S_e assigned VG's parameters 
       and a value for soil suction $\psi$ 
    """
    m =1-1/n
    return thetaR+(thetaS-thetaR)/( 1+(alpha*np.abs(psi))**n )**(m)


theta = []
for i in range(0,np.size(psiIC[:])-1):
    theta=np.append(theta, VG(psiIC[i],par1SWRC[i],par2SWRC[i],thetaS[i],thetaR[i]) ) 

In [7]:
airVolume = (thetaS[:]-theta)*deltaZ[:]
#airVolume

In [8]:
## unit [m]
totalAirVolume = np.sum(airVolume)
totalAirVolume

0.20171816882731133

## Read output file
Reading the simulation output file allows to konow the cumulative rainfall height and the water ponding depth

In [9]:
os.chdir("C:\\Users\\Niccolo\\OMS\\OMS_Project_Richards1D_Newest\\output")

os.listdir()

['Clay_noPonding_2mmRain_BottomImpervious.nc',
 'Clay_noPonding_noRain_BottomDirichlet.nc',
 'Clay_noPonding_noRain_BottomNeumann.nc',
 'Sand_01Ponding_noRain_BottomDirichlet.nc',
 'Sand_01Ponding_noRain_BottomFreeDrainage.nc']

In [10]:
##########
##########

# open netCDF file for reading.

ncfile = Dataset('Clay_noPonding_2mmRain_BottomImpervious.nc','r') 
 
print ('*** SUCCESS reading')

print('\n ***FILE INFO:\n')
print(ncfile)

## other usefull commands:
#print (ncfile.dimensions['time'])
#print (ncfile.file_format)
#print (ncfile.dimensions.keys())
print (ncfile.variables.keys())
#print (ncfile.variables['psi'])


depths = ncfile.variables['depth']
print('\n ***DEPTHS INFO:\n')
print(depths)

dualDepths = ncfile.variables['dual_depth']
print('\n ***DUAL DEPTH INFO:\n')
print(dualDepths)

time = ncfile.variables['time']
print('\n ***TIME INFO:\n')
print(time)

psi = ncfile.variables['psi']
print('\n ***PSI INFO:\n')
print(psi)

theta = ncfile.variables['water_heigth']
print('\n ***THETA INFO:\n')
print(theta)

iC = ncfile.variables['psiIC']
print('\n ***INITIAL CONDITION INFO:\n')
print(iC)

velocities = ncfile.variables['velocities']
print('\n ***VELOCITIES INFO:\n')
print(velocities)

error = ncfile.variables['error']
print('\n ***ERROR INFO:\n')
print(error)

topBC = ncfile.variables['topBC']
print('\n ***topBC INFO:\n')
print(topBC)

bottomBC = ncfile.variables['bottomBC']
print('\n ***bottomBC INFO:\n')
print(bottomBC)


*** SUCCESS reading

 ***FILE INFO:

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Description_of_the_problem: 1 layer of clay 
 Grid input file: C:\Users\Niccolo\OMS\OMS_Project_Richards1D_Newest//data/Grid_NEtCDF/Clay_noPonding.nc 
 TopBC input file: C:\Users\Niccolo\OMS\OMS_Project_Richards1D_Newest//data/Timeseries/TestAll_2.csv 
 BottomBC input file: C:\Users\Niccolo\OMS\OMS_Project_Richards1D_Newest//data/Timeseries/TestAll_0.csv, timeDelta = 10, interface k: mean
    dimensions(sizes): depth(321), dualDepth(321), time(577)
    variables(dimensions): float64 depth(depth), float64 dual_depth(dualDepth), int32 time(time), float64 psi(time,depth), float64 psiIC(depth), float64 water_heigth(time,depth), float64 velocities(time,dualDepth), float64 error(time), float64 topBC(time), float64 bottomBC(time)
    groups: 

odict_keys(['depth', 'dual_depth', 'time', 'psi', 'psiIC', 'water_heigth', 'velocities', 'error', 'topBC', 'bottomB

In [11]:
## ponding depth at soil surface at the end of the simulation [m]

PondingDepth = psi[np.size(time[:])-1,np.size(depths[:])-1]
PondingDepth

-0.0020406709430580766

In [12]:
## cumulative rainfall height [mm]
mmRainfall = np.sum(topBC[:])
mmRainfall

1154.0

In [13]:
estimatedPonding = mmRainfall-totalAirVolume*1000
estimatedPonding

952.28183117268873

In [14]:
relativeError = (PondingDepth*1000-estimatedPonding)/estimatedPonding
relativeError

-1.0021429275202542

In [13]:
ncfile.close()